# Segmenting and Clustering Neighborhoods in Toronto

##### Thank You for reviewing my work!
##### This Notebook is for Answer 3 of:
##### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [48]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Scrap List of postal codes of Canada wiki page content by using BeautifulSoup

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

#### Find the first table on the Wikipedia page and iterate through tags for required information

In [3]:
table_can_zipinfo = soup.find('table')
colvals = table_can_zipinfo.find_all('td')

elem_cnt = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

#### Build the dataframe from the list of values

In [4]:
df_can_postcode = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_can_postcode.columns = ['Postcode', 'Borough', 'Neighborhood']

#### Cleansing and Transforming the data
###### 1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
###### 2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
df_can_postcode.drop(df_can_postcode[df_can_postcode['Borough'] == 'Not assigned'].index, inplace=True)
df_can_postcode.loc[df_can_postcode.Neighborhood == 'Not assigned', "Neighborhood"] = df_can_postcode.Borough
df_can_postcode.reset_index(drop=True,inplace=True)

#### Visualizing first few rows of the dataframe 

In [6]:
df_can_postcode.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Printing the number of rows of your dataframe

In [7]:
df_can_postcode.shape

(212, 3)

#### Read the Geospatial csv file and inner join it with original dataframe

In [8]:
df_latlong = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlong.columns = ['Postcode', 'Latitude', 'Longitude']

#### Having a quick look at the latitude and longitude data

In [9]:
df_latlong.head(3)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


### Joining the latitude and longitude data with Original Dataframe

In [10]:
df_toronto = pd.merge(df_can_postcode, df_latlong, on=['Postcode'], how='inner')

In [11]:
df_toronto.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


#### Displaying shape of our dataframe

In [12]:
neighborhoods = df_toronto [['Borough', 'Neighborhood', 'Latitude', 'Longitude']].copy()
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 212 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto

In [13]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="trn_explorer")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_x, longitude_y))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        ).add_to(map_toronto)  
    
map_toronto

### Simplify the above map and segment and cluster only the neighborhoods in  Downtown Toronto. So let's slice the original dataframe and create a new dataframe of the Downtown Toronto data

In [37]:
df_toronto_new = neighborhoods[neighborhoods['Borough'].str.contains("Downtown Toronto")].reset_index(drop=True)
df_toronto_new.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Regent Park,43.654260,-79.360636
2,Downtown Toronto,Ryerson,43.657162,-79.378937
3,Downtown Toronto,Garden District,43.657162,-79.378937
4,Downtown Toronto,St. James Town,43.651494,-79.375418


### Lets Explore Downtown Toronto, Canada
 I will begin by getting the Geo Co-ordinates of Downtown Toronto

In [38]:
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent="dtCan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto, Canada are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Downtown Toronto, Canada are 43.655115, -79.380219.


Let's visualize Downtown Toronto neighbourhoods in the Map

In [40]:
# create map of Downtown Toronto using latitude and longitude values
map_dttoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto_new['Latitude'], df_toronto_new['Longitude'], df_toronto_new['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_dttoronto)  
    
map_dttoronto

### Define Foursquare Credentials and Version

In [41]:
CLIENT_ID = 'NLXCKQ23MN2GCTBG4TRVXSFXYTVUPRPS120IIZIGNQ4CIYTP' # your Foursquare ID
CLIENT_SECRET = 'LPDQZSSGUP3DSYQ4X3VKLKFZMYTIMCZ1MJFM1K43XMLS4TJK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NLXCKQ23MN2GCTBG4TRVXSFXYTVUPRPS120IIZIGNQ4CIYTP
CLIENT_SECRET:LPDQZSSGUP3DSYQ4X3VKLKFZMYTIMCZ1MJFM1K43XMLS4TJK


#### Get the neighborhood's name

In [42]:
df_toronto_new.loc[0, 'Neighborhood']

'Harbourfront'

#### Get the neighborhood's latitude and longitude values

In [43]:
neighborhood_latitude = df_toronto_new.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_new.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto_new.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


### Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters

Create the GET request URL

In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=NLXCKQ23MN2GCTBG4TRVXSFXYTVUPRPS120IIZIGNQ4CIYTP&client_secret=LPDQZSSGUP3DSYQ4X3VKLKFZMYTIMCZ1MJFM1K43XMLS4TJK&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

### Sending the GET request and examine the Results

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4c8aec351e3d1df9988979'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

#### We know that all the information is in the "items" key. Before we proceed, we will borrow the "get_category_type" function from the Foursquare lab

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json and structure it into a pandas dataframe

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


### Check how many venues were returned by Foursquare

In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown Toronto

#### Create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Running the above function on each neighborhood and create a new dataframe called *DowntownTorontoVenues*.

In [52]:
DowntownTorontoVenues = getNearbyVenues(names=df_toronto_new['Neighborhood'],
                                   latitudes=df_toronto_new['Latitude'],
                                   longitudes=df_toronto_new['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [54]:
print(DowntownTorontoVenues.shape)
DowntownTorontoVenues.head()

(2499, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### Venues returned for each neighborhood

In [58]:
DowntownTorontoVenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56
CN Tower,14,14,14,14,14,14
Cabbagetown,48,48,48,48,48,48
Central Bay Street,82,82,82,82,82,82
Chinatown,100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,87,87,87,87,87,87


#### Finding out how many unique categories can be curated from all the returned venues

In [59]:
print('There are {} uniques categories.'.format(len(DowntownTorontoVenues['Venue Category'].unique())))

There are 206 uniques categories.


## Analyze Each Neighborhood

In [60]:
# one hot encoding
dtToronto_onehot = pd.get_dummies(DowntownTorontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtToronto_onehot['Neighborhood'] = DowntownTorontoVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dtToronto_onehot.columns[-1]] + list(dtToronto_onehot.columns[:-1])
dtToronto_onehot = dtToronto_onehot[fixed_columns]

dtToronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
dtToronto_onehot.shape

(2499, 206)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [64]:
dtToronto_grouped = dtToronto_onehot.groupby('Neighborhood').mean().reset_index()
dtToronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,CN Tower,0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000
6,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.00,0.00,0.00,0.050000,0.000000,0.050000,0.010000,0.000000,0.000000
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Church and Wellesley,0.011494,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.011494,0.011494,0.000000,0.011494,0.000000
9,Commerce Court,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000


In [66]:
dtToronto_grouped.shape

(36, 206)

#### Print each neighborhood along with the top 5 most common venues

In [68]:
num_top_venues = 5

for hood in dtToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dtToronto_grouped[dtToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Bathurst Quay----
              venue  freq
0   Airport Service  0.14
1  Airport Terminal  0.14
2    Airport Lounge  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Restaurant  0.05
3          Café  0.04
4           Pub  0.04


----CN Tower----
              venue  freq
0   Airport Service  0.14
1  Airport Terminal  0.14
2    Airport Lounge  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


----Cabbagetown----
         venue  freq
0   Restaurant  0.08
1  Coffee Shop  0.08
2          Pub  0.04
3         Café  0.04
4       Market  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.07
2  Italian Restaurant  0.06
3        Bu

#### Organising the data into a *pandas* dataframe

In [69]:
## function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
## create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dtToronto_grouped['Neighborhood']

for ind in np.arange(dtToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Clothing Store,Bakery,Bar
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Sculpture Garden,Boutique,Airport,Airport Food Court,Airport Gate
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Pub,Café,Steakhouse,Cheese Shop,Italian Restaurant,Beer Bar,Seafood Restaurant
3,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Sculpture Garden,Boutique,Airport,Airport Food Court,Airport Gate
4,Cabbagetown,Restaurant,Coffee Shop,Bakery,Convenience Store,Pub,Café,Market,Pizza Place,Italian Restaurant,Park


## Cluster the Neighborhoods

### Running k-means to Cluster the neighbourhoods into 5 Clusters

In [71]:
# set number of clusters
kclusters = 5

dtToronto_grouped_clustering = dtToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 2, 0, 0, 3, 4, 0, 0], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtToronto_merged = df_toronto_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dtToronto_merged = dtToronto_merged1.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dtToronto_merged.head() 

ValueError: cannot insert Cluster Labels, already exists

## Visualize the resulting clusters

In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtToronto_merged['Latitude'], dtToronto_merged['Longitude'], dtToronto_merged['Neighborhood'], dtToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Thank You for Reviewing my work!!